In [1]:
import os
import requests
from bs4 import BeautifulSoup
import json 
from dotenv import load_dotenv
from typing import List
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key wrking fine")
else:
    print("Check api key again")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key wrking fine


In [3]:
header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

class Website:
    def __init__(self,url):
        self.url=url
        response=requests.get(url, headers=header)
        self.body=response.content
        soup=BeautifulSoup(self.body, 'html.parser')
        self.title=soup.title.string if soup.title else "No title found"
        if soup.body:
            for faaltu in soup.body(['script', 'style','img','input']):
                faaltu.decompose()
            self.text=soup.body.get_text(separator='\n', strip=True)
        else:
            self.text=""
        links=[link.get('href') for link in soup.find_all('a')]
        self.links=[link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [30]:
system_prompt="You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"

system_prompt+='You should respond in JSON as in this example:'
system_prompt+="""
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [31]:
def get_user_prompt(website):
    user_prompt=f'Here is the list of links on the website of {website.url} - '
    user_prompt+="please decide which of these are relevant web links  for a brochure about the company, respond with the full https URL in JSON format. \
        do not include Terms of Service, Privacy, email links.\n"
    user_prompt+="Links (some might be relative links):\nGive the output strictly in python list form"
    user_prompt+="\n".join(website.links)

    return user_prompt

In [32]:
def get_links(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content":system_prompt},
            {"role":"user", "content": get_user_prompt(website)}
        ],

            response_format={"type":"json_object"}
    )
    result=response.choices[0].message.content
    return json.loads(result)
    

In [33]:
import time

def retry(url, retries=5, delay=2):
    for attempt in range(retries):
        try:
            return Website(url).get_contents()  
        except (ConnectTimeout, TimeoutError, Exception) as e:
            print(f"Error occurred for {url}: {e}. Attempt {attempt + 1} of {retries}.")
            time.sleep(delay)  
    print(f"Skipping {url} after {retries} failed attempts.")
    return f"Failed to retrieve contents for {url}\n"

def get_details(url):
    result = "Landing page:\n"
    result += retry(url)  
    links = get_links(url)
    print("Found links:", links)
    
    key_to_check = next(iter(links), None) 
    
    if key_to_check == 'links':
        for link in links["links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    elif key_to_check == 'relevant_links':
        for link in links["relevant_links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    else:
        result += "\nNo relevant links found.\n"
    
    return result


In [34]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a website ( it can be company website, college club website, someone's portfolio) so be prepared \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [39]:
def brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a professional brief brochure of the website in markdown.At footer always give the contacts link of their social media handles (except phone number) if available\n"
    user_prompt += get_details(url)
    user_prompt = user_prompt[:5_000] 
    return user_prompt

In [36]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [37]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("RoboVitics", "https://robovitics.in/")

Found links: {'relevant_links': ['https://robovitics.in/#about-page', 'https://robovitics.in/#events-page', 'https://robovitics.in/#projects-page', 'https://robovitics.in/#orcus-page', 'https://robovitics.in/#kartavya-page', 'https://robovitics.in/#sponsors-page', 'https://robovitics.in/#memories', 'https://twitter.com/robo_vit', 'https://www.facebook.com/robovitics', 'https://www.instagram.com/robovitics/', 'https://www.linkedin.com/company/robovitics/', 'https://www.youtube.com/channel/UCFiwOI-W5b06NweratR-RdA', 'https://github.com/RoboVITics']}


# RoboVitics: Where Innovation Meets Imagination!

Welcome to RoboVitics, the **robotics club** that doesn't just aim for the stars; we send our robots through the stratosphere! Based at **Vellore Institute of Technology (VIT)**, we’re not just a group of tech enthusiasts - we’re a *force of nature* in the world of robotics. 

### 📚 **About Us**
Founded in 1984 (way before selfies were a thing!), our mission is simple: “A place to learn, a chance to grow.” We believe innovation is when imagination meets ambition—unless you're imagining being a couch potato on a Saturday, then we can't help you there.

Our club started as a small group of passionate techies and has grown into a powerhouse representing the best of VIT worldwide. We offer a plethora of free workshops and seminars that turn your technological dreams into reality. 🦾✨

### 🎉 **Events (and a Little Chaos)**
Get ready for **equilibrium** and **chaos** all at once!

- **Equinox**: 36 hours of tech madness! Bring your ideas, creativity, and maybe a snack or two. We have mentors and resources to turn wild ideas into concrete projects.
  
- **RoboWars**: Battlebots like you’ve never seen! Witness the metallic mayhem as robots duke it out for the glory of RoboWars Champion. Spoiler: sparks WILL fly. ⚔️🤖

- **Hands-On Robotics**: Build fully functional Arduino robots and unleash them upon the world – or just the lab table. Perfect for both budding engineers and those looking for robot sidekicks!

- **Vortex360**: A CAD modeling hackathon where designers connect and let their creativity flow like a freshly opened soda can. 

### 🚀 **Projects** 
Our pride, our joy! Our members have collaborated to create some incredible stuff like:
- **Spiderbot**: An 8-legged robot perfect for tasks like bomb defusing or landscaping your front yard (just kidding!). It's a nimble creature designed for rough terrains, ready to take on the ‘tough’ tasks humans avoid. 

- **Robotic Arm**: Think of it as an extra limb—but one that doesn’t need coffee breaks.

### 👩‍💻 **Careers & Club Culture**
Join us, and *become a RoboVITian*! We promise not just a space to innovate but a whole lot of fun. Our culture celebrates collaboration, learning, and the occasional pizza party (who can innovate without fuel?!). 

You’ll experience your best years with a vibrant community that thrives on curiosity and growth. So, if you've got a passion for tech (or just like to hang around nerdy geniuses), we want you on board! 

### Ready to Join?
If you’re curious about going from a mere human to a tech genius or just want to see cool robots, reach out! 

Visit us:
- [Instagram](#)
- [Twitter](#)
- [LinkedIn](#)

### RoboVitics: Building Robots & Making Memories! 🚀🤖

Come join the revolution—where we grow, learn, and occasionally laugh at our own robot fails!

NoneType

In [43]:
type(output)

NoneType